In [1]:
import pandas as pd

In [2]:
df_location = pd.read_stata('../dataset/classifications_data/location.dta')
df_location.head(10)

,location_id,location_code,location_name_short_en,level,parent_id
0,0,ABW,Aruba,country,356.0
1,1,AFG,Afghanistan,country,353.0
2,2,AGO,Angola,country,352.0
3,3,AIA,Anguilla,country,356.0
4,4,ALB,Albania,country,355.0
5,5,AND,Andorra,country,355.0
6,6,ANT,Netherlands Antilles,country,356.0
7,7,ARE,United Arab Emirates,country,353.0
8,8,ARG,Argentina,country,357.0
9,9,ARM,Armenia,country,353.0


In [3]:
df_data_class = pd.read_stata('../dataset/classifications_data/sitc_product.dta')
df_data_class.head(10)

,product_id,sitc_product_code,sitc_product_name_short_en,level,parent_id
0,0,0,Food,section,NaN
1,1,1,Beverages,section,NaN
2,2,2,Crude materials,section,NaN
3,3,3,Fuels,section,NaN
4,4,4,Vegetable oils,section,NaN
5,5,5,Chemicals,section,NaN
6,6,6,Material manufacturers,section,NaN
7,7,7,Machinery and vehicles,section,NaN
8,8,8,Other manufacturers,section,NaN
9,9,9,Unspecified,section,NaN


In [4]:
folder_path = '../dataset/dataverse_files/'
country_partner_sitc_4digit = 'country_partner_sitcproduct4digit_year_{}.csv'
country_partner_sitc_2digit = 'country_partner_sitcproduct2digit_year.csv'
country_partner_sitc_section = 'country_partner_sitcproductsection_year.csv'
country_sitc_2digit = 'country_sitcproduct2digit_year.csv'
country_sitc_4digit = 'country_sitcproduct4digit_year.csv'
country_sitc_section = 'country_sitcproductsection_year.csv'
sitc_2digit=2
sitc_4digit=4

def get_data(country_partner=True, sitc_digit=4, year=2019):
    """
    Creates a Dataframe for a specified SITC dataset

    Args:
        country_partner: If True, dataset with trades between countries and partners are selected
        sitc_digit: 4 for SITC-4 digit products, 2 for SITC-2 digit products, otw SITC product section
        year: Year between 1962 and 2019 for the country-partner SITC-4 digit products
    Returns:
        Dataframe of the selected dataset
    """
    path = folder_path
    if country_partner:
        if sitc_digit==sitc_4digit:
            path += country_partner_sitc_4digit.format(year)
        elif sitc_digit==sitc_2digit:
            path += country_partner_sitc_2digit
        else:
            path += country_partner_sitc_section
    else:
        if sitc_digit==sitc_4digit:
            path += country_sitc_4digit
        elif sitc_digit==sitc_2digit:
            path += country_sitc_2digit
        else:
            path += country_sitc_section

    return pd.read_csv(path)

In [5]:
df_2019 = get_data()

In [6]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4085596 entries, 0 to 4085595
Data columns (total 11 columns):
location_id          int64
partner_id           int64
product_id           int64
year                 float64
export_value         int64
import_value         int64
sitc_eci             float64
sitc_coi             float64
location_code        object
partner_code         object
sitc_product_code    int64
dtypes: float64(3), int64(6), object(2)
memory usage: 342.9+ MB


In [7]:
df_2019.head(10)

,location_id,partner_id,product_id,year,export_value,import_value,sitc_eci,sitc_coi,location_code,partner_code,sitc_product_code
0,0,1,670,2019.0,2124,0,1.001882,0.056012,ABW,AFG,230
1,7,1,670,2019.0,88420,0,-0.435528,-0.954248,ARE,AFG,230
2,61,1,670,2019.0,19988,0,2.065511,-2.215808,DEU,AFG,230
3,64,1,670,2019.0,158738,0,1.098704,1.306768,DNK,AFG,230
4,162,1,670,2019.0,123476,0,1.110004,1.409379,NLD,AFG,230
5,177,1,670,2019.0,6684,0,1.079311,1.300873,POL,AFG,230
6,224,1,670,2019.0,131030,0,0.610807,2.321267,TUR,AFG,230
7,1,7,670,2019.0,0,88420,-1.057238,-0.809024,AFG,ARE,230
8,8,7,670,2019.0,176167,0,-0.243857,0.257674,ARG,ARE,230
9,14,7,670,2019.0,370111,81174,-0.454162,-0.278427,AUS,ARE,230


In [8]:

df_2019[['export_value', 'import_value']].describe()

,export_value,import_value
count,4.085596e+06,4.085596e+06
mean,4.463191e+06,4.463191e+06
std,1.425441e+08,1.425441e+08
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,9.335000e+03,9.335000e+03
75%,1.703570e+05,1.703570e+05
max,1.272548e+11,1.272548e+11


In [9]:
print('Countries in 2019 country-partner SITC-4 digit products dataset: {} out of {}'.format(len(df_2019['location_id'].unique()), len(df_location['location_id'].unique())))
print('Products in 2019 country-partner SITC-4 digit products dataset: {}'.format(len(df_2019['product_id'].unique())))


Countries in 2019 country-partner SITC-4 digit products dataset: 235 out of 258
Products in 2019 country-partner SITC-4 digit products dataset: 766


## Strongest countries according to descending export value

In [10]:
df_country_names = df_location[['location_id', 'location_name_short_en']].astype({'location_id': int}, errors='raise')
df_highest_exports = df_2019.groupby('location_id', as_index=False).agg({'export_value':'sum'}).sort_values('export_value', ascending=False)
df_highest_exports = df_highest_exports.merge(df_country_names, on='location_id')
exports_sum = df_highest_exports['export_value'].sum()
df_highest_exports['Percentage'] = df_highest_exports['export_value'] * 100 / exports_sum

df_highest_exports.head(10)


,location_id,export_value,location_name_short_en,Percentage
0,43,2455821372827,China,13.467775
1,231,1632206655429,United States of America,8.951055
2,61,1492780104757,Germany,8.186437
3,114,717978619544,Japan,3.937409
4,77,554501869931,France,3.040900
5,162,545136055834,Netherlands,2.989537
6,121,540193910526,South Korea,2.962434
7,111,537607163298,Italy,2.948249
8,138,486154177275,Mexico,2.666079
9,81,464857429180,United Kingdom,2.549288


## Strongest countries according to descending import value

In [11]:
df_highest_imports = df_2019.groupby('location_id', as_index=False).agg({'import_value':'sum'}).sort_values('import_value', ascending=False)
df_highest_imports = df_highest_imports.merge(df_country_names, on='location_id')
imports_sum = df_highest_imports['import_value'].sum()
df_highest_imports['Percentage'] = df_highest_imports['import_value'] * 100 / imports_sum
df_highest_imports.head(10)

,location_id,import_value,location_name_short_en,Percentage
0,231,2333016626976,United States of America,12.794311
1,43,1643417201652,China,9.012534
2,61,1165700201526,Germany,6.392724
3,81,658130564040,United Kingdom,3.609201
4,77,638637545124,France,3.502301
5,114,623265542910,Japan,3.418001
6,97,593736031925,Hong Kong,3.256061
7,162,556223571410,Netherlands,3.050341
8,121,465677825198,South Korea,2.553787
9,39,460740627404,Canada,2.526711


## Strongest partners according to descending export/import value

In [12]:
df_highest_partners = df_2019.groupby(['location_id', 'partner_id'], as_index=False).agg({'export_value':'sum'})
df_highest_partners = df_highest_partners.merge(df_country_names, on='location_id')
df_highest_partners = df_highest_partners.merge(df_country_names.rename(columns = {'location_id':'partner_id'}), on='partner_id')
df_highest_partners = df_highest_partners.sort_values('export_value', ascending=False)
df_highest_partners.rename(columns = {'export_value':'value','location_name_short_en_x':'export_country', 'location_name_short_en_y':'import_country'}, inplace = True)

value_sum = df_highest_partners['value'].sum()
df_highest_partners['Percentage'] = df_highest_partners['value'] * 100 / imports_sum

df_highest_partners.head(10)

,location_id,partner_id,value,export_country,import_country,Percentage
11320,43,231,387348497156,China,United States of America,2.124227
11410,138,231,361292099796,Mexico,United States of America,1.981333
11316,39,231,330853780633,Canada,United States of America,1.814409
1672,231,39,288655180128,United States of America,Canada,1.582991
4937,43,97,278149267556,China,Hong Kong,1.525376
7674,231,138,255492473497,United States of America,Mexico,1.401126
12103,7,250,201696250691,United Arab Emirates,Undeclared Countries,1.106106
6542,43,114,145552998211,China,Japan,0.798216
11386,114,231,145483251390,Japan,United States of America,0.797833
2174,121,43,143445067089,South Korea,China,0.786656


## Highest traded SITC-4 digit products according to descending value

In [13]:
df_highest_products = df_2019.groupby('product_id', as_index=False).agg({'export_value':'sum'}).sort_values('export_value', ascending=False)

df_highest_products = df_highest_products.merge(df_data_class[['product_id', 'sitc_product_name_short_en']], on='product_id')
df_highest_products['Percentage'] = df_highest_products['export_value'] * 100 / exports_sum
df_highest_products.rename(columns = {'export_value':'value'}, inplace = True)


df_highest_products.head(10)

,product_id,value,sitc_product_name_short_en,Percentage
0,865,1090531439917,Crude petroleum and oils obtained from bitumin...,5.980497
1,866,740376437519,"Petroleum products, refined",4.060240
2,1314,732562848187,Passenger motor vehicles (excluding buses),4.017390
3,1432,676133162948,"Special transactions, commodity not classified...",3.707928
4,1307,635348789903,Electronic microcircuits,3.484266
5,1286,384751586283,"Television, radio-broadcasting; transmitters, etc",2.109986
6,1321,378064157802,"Other parts and accessories, for vehicles of h...",2.073312
7,944,376685692337,Medicaments (including veterinary medicaments),2.065752
8,1436,329225666286,"Gold, non-monetary (excluding gold ores and co...",1.805480
9,1276,226799654109,"Parts, nes of and accessories for machines of ...",1.243774


In [14]:
import json
 
# Opening JSON file
f = open('../dataset/classifications_data/ne_110m_admin_0_countries.geojson')
 
# returns JSON object as
# a dictionary
data = json.load(f)
f.close()

In [ ]:
data["features"]

In [ ]:
for i in data["features"]:
    print(i["properties"]["ISO_A3"])
    print(i["properties"]["ISO_A2"]) 
    print(i["properties"]["ADMIN"])
    print("------------")

In [17]:
def fix_country_codes(name):
    if name == "France":
        return ["FR","FRA"]
    elif name == "Norway":
        return ["NO","NOR"]
    elif name == "Kosovo":
        return ["XK","XKX"]        
    else:
        print(name)
        return ["-99","-99"]

In [18]:
countries = pd.read_csv('../dataset/classifications_data/countries.csv') 
countries = countries.rename(columns={"country": "ISO_A2"})
countries["ISO_A3"] = 0
for i in data["features"]: 
    if i["properties"]["ISO_A3"] == "-99":
        a2, a3 = fix_country_codes(i["properties"]["ADMIN"])
        countries.loc[countries.ISO_A2 == a2, 'ISO_A3'] = a3        
    else:
        countries.loc[countries.ISO_A2 == i["properties"]["ISO_A2"], 'ISO_A3'] = i["properties"]["ISO_A3"]
countries = countries.drop(countries[countries.ISO_A3 == 0].index)
countries["location_id"] = -1
for index, row in df_location.iterrows():    
    countries.loc[countries.ISO_A3 == row['location_code'], 'location_id'] = row['location_id']
    countries.loc[countries.ISO_A3 == row['location_code'], 'name'] = row['location_name_short_en']
countries

Northern Cyprus
Somaliland


,ISO_A2,latitude,longitude,name,ISO_A3,location_id
1,AE,23.424076,53.847818,United Arab Emirates,ARE,7
2,AF,33.939110,67.709953,Afghanistan,AFG,1
5,AL,41.153332,20.168331,Albania,ALB,4
6,AM,40.069099,45.038189,Armenia,ARM,9
8,AO,-11.202692,17.873887,Angola,AGO,2
...,...,...,...,...,...,...
239,XK,42.602636,20.902977,Kosovo,XKX,-1
240,YE,15.552727,48.516388,Yemen,YEM,243
242,ZA,-30.559482,22.937506,South Africa,ZAF,246
243,ZM,-13.133897,27.849332,Zambia,ZMB,247


In [19]:
countries[countries.location_id == -1]

,ISO_A2,latitude,longitude,name,ISO_A3,location_id
178,PR,18.220833,-66.590149,Puerto Rico,PRI,-1
239,XK,42.602636,20.902977,Kosovo,XKX,-1


In [20]:
data_df = df_2019.drop(["product_id","year","import_value","sitc_eci","sitc_coi","location_code","partner_code","sitc_product_code"],axis = 1)
data_df

,location_id,partner_id,export_value
0,0,1,2124
1,7,1,88420
2,61,1,19988
3,64,1,158738
4,162,1,123476
...,...,...,...
4085591,77,155,0
4085592,231,155,0
4085593,246,155,1708
4085594,162,109,1487


In [21]:
data_df = data_df.groupby(["location_id","partner_id"], sort=True).sum().reset_index()
data_df = data_df.drop(data_df[~data_df.location_id.isin(countries.location_id)].index)
data_df = data_df.drop(data_df[~data_df.partner_id.isin(countries.location_id)].index)
data_df

,location_id,partner_id,export_value
72,1,7,139235524
73,1,8,0
75,1,14,1528993
76,1,15,1023265
77,1,16,0
...,...,...,...
25702,249,240,12691587
25704,249,243,16652214
25705,249,246,581332324
25706,249,247,2709198


In [22]:
dummy = data_df.groupby('location_id').apply(lambda x : x.nlargest(20, "export_value")).reset_index(drop = True)
dummy

,location_id,partner_id,export_value
0,1,168,560615684
1,1,104,464483581
2,1,7,139235524
3,1,231,30358140
4,1,43,29315585
...,...,...,...
3390,249,39,2596081214
3391,249,77,1898160355
3392,249,111,1838950384
3393,249,7,1488216133


In [23]:
dummy = dummy.astype({"location_id": str})
dummy = dummy.astype({"partner_id": str})

In [24]:
dummy["main_code"] = -1
dummy["main_name"] = -1
dummy["main_lat"] = -1
dummy["main_lon"] = -1
dummy["partner_code"] = -1
dummy["partner_name"] = -1
dummy["partner_lat"] = -1
dummy["partner_lon"] = -1

for index, row in countries.iterrows():    
    dummy.loc[dummy.location_id == row['location_id'], 'main_code'] = row['ISO_A3']
    dummy.loc[dummy.location_id == str(row['location_id']), 'main_lat'] = row['latitude']
    dummy.loc[dummy.location_id == str(row['location_id']), 'main_lon'] = row['longitude']
    dummy.loc[dummy.location_id == str(row['location_id']), 'main_name'] = row['name']
    
    dummy.loc[dummy.partner_id == str(row['location_id']), 'partner_code'] = row['ISO_A3']
    dummy.loc[dummy.partner_id == str(row['location_id']), 'partner_lat'] = row['latitude']
    dummy.loc[dummy.partner_id == str(row['location_id']), 'partner_lon'] = row['longitude']
    dummy.loc[dummy.partner_id == str(row['location_id']), 'partner_name'] = row['name']
    
dummy = dummy.drop(["location_id","partner_id"],axis = 1)   
dummy

,export_value,main_code,main_name,main_lat,main_lon,partner_code,partner_name,partner_lat,partner_lon
0,560615684,AFG,Afghanistan,33.93911,67.709953,PAK,Pakistan,30.375321,69.345116
1,464483581,AFG,Afghanistan,33.93911,67.709953,IND,India,20.593684,78.962880
2,139235524,AFG,Afghanistan,33.93911,67.709953,ARE,United Arab Emirates,23.424076,53.847818
3,30358140,AFG,Afghanistan,33.93911,67.709953,USA,United States of America,37.090240,-95.712891
4,29315585,AFG,Afghanistan,33.93911,67.709953,CHN,China,35.861660,104.195397
...,...,...,...,...,...,...,...,...,...
3390,2596081214,TWN,Taiwan,23.69781,120.960515,CAN,Canada,56.130366,-106.346771
3391,1898160355,TWN,Taiwan,23.69781,120.960515,FRA,France,46.227638,2.213749
3392,1838950384,TWN,Taiwan,23.69781,120.960515,ITA,Italy,41.871940,12.567380
3393,1488216133,TWN,Taiwan,23.69781,120.960515,ARE,United Arab Emirates,23.424076,53.847818


In [25]:
#Add percentage for opacity
sums = dummy.groupby("main_code")["export_value"].sum()
dummy["percentage"] = -1
for index, row in dummy.iterrows():
     dummy.loc[index, "percentage"] = row["export_value"] / sums[row["main_code"]]
dummy

,export_value,main_code,main_name,main_lat,main_lon,partner_code,partner_name,partner_lat,partner_lon,percentage
0,560615684,AFG,Afghanistan,33.93911,67.709953,PAK,Pakistan,30.375321,69.345116,0.412676
1,464483581,AFG,Afghanistan,33.93911,67.709953,IND,India,20.593684,78.962880,0.341912
2,139235524,AFG,Afghanistan,33.93911,67.709953,ARE,United Arab Emirates,23.424076,53.847818,0.102493
3,30358140,AFG,Afghanistan,33.93911,67.709953,USA,United States of America,37.090240,-95.712891,0.022347
4,29315585,AFG,Afghanistan,33.93911,67.709953,CHN,China,35.861660,104.195397,0.021580
...,...,...,...,...,...,...,...,...,...,...
3390,2596081214,TWN,Taiwan,23.69781,120.960515,CAN,Canada,56.130366,-106.346771,0.009692
3391,1898160355,TWN,Taiwan,23.69781,120.960515,FRA,France,46.227638,2.213749,0.007087
3392,1838950384,TWN,Taiwan,23.69781,120.960515,ITA,Italy,41.871940,12.567380,0.006865
3393,1488216133,TWN,Taiwan,23.69781,120.960515,ARE,United Arab Emirates,23.424076,53.847818,0.005556


In [26]:
dummy[dummy["main_code"] == "USA"]

,export_value,main_code,main_name,main_lat,main_lon,partner_code,partner_name,partner_lat,partner_lon,percentage
3195,288655180128,USA,United States of America,37.09024,-95.712891,CAN,Canada,56.130366,-106.346771,0.226848
3196,255492473497,USA,United States of America,37.09024,-95.712891,MEX,Mexico,23.634501,-102.552784,0.200786
3197,105880174550,USA,United States of America,37.09024,-95.712891,CHN,China,35.861660,104.195397,0.083209
3198,74131563122,USA,United States of America,37.09024,-95.712891,JPN,Japan,36.204824,138.252924,0.058258
3199,68383682361,USA,United States of America,37.09024,-95.712891,GBR,United Kingdom,55.378051,-3.435973,0.053741
3200,60833709846,USA,United States of America,37.09024,-95.712891,DEU,Germany,51.165691,10.451526,0.047808
3201,56528821767,USA,United States of America,37.09024,-95.712891,KOR,South Korea,35.907757,127.766922,0.044425
3202,50468347798,USA,United States of America,37.09024,-95.712891,NLD,Netherlands,52.132633,5.291266,0.039662
3203,42755973066,USA,United States of America,37.09024,-95.712891,BRA,Brazil,-14.235004,-51.925280,0.033601
3204,38381117644,USA,United States of America,37.09024,-95.712891,FRA,France,46.227638,2.213749,0.030163


In [27]:
result = {}
for code in dummy.main_code.unique():
    result[code] = dummy[dummy.main_code == code].to_numpy().tolist()

In [28]:
with open("geo_export.json", "w") as outfile:
    json.dump(result, outfile)

In [29]:
# Opening JSON file
f = open("geo_export.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)
f.close() 